In [1]:
def divide(dividend: int, divisor: int) -> int:
    MAX_INT = 2147483647
    MIN_INT = -2147483648
    
    if dividend == MIN_INT and divisor == -1:
        return MAX_INT
    
    negative = (dividend < 0) != (divisor < 0)
    
    abs_dividend = abs(dividend)
    abs_divisor = abs(divisor)
    
    quotient = 0
    
    while abs_dividend >= abs_divisor:
        temp_divisor = abs_divisor
        multiple = 1
        
        while abs_dividend >= (temp_divisor << 1):
            temp_divisor <<= 1
            multiple <<= 1
            
        abs_dividend -= temp_divisor
        quotient += multiple
        
    if negative:
        return -quotient
    else:
        return quotient

The LeetCode problem 29, "Divide Two Integers," is a classic challenge that requires performing integer division without using the built-in multiplication, division, or modulo operators. The core goal is to find the integer quotient of two given integers, the `dividend` and the `divisor`, while strictly adhering to the 32-bit signed integer range $[-2^{31}, 2^{31} - 1]$ for the result.

---

### **Handling Signs and Edge Cases**

The first step in solving this problem efficiently is to simplify the arithmetic by working with positive numbers. We should determine the sign of the final quotient by checking if the `dividend` and `divisor` have different signs (exclusive OR). Once the final sign is determined, both numbers are converted to their absolute values. This prevents sign manipulation within the main division loop. A critical edge case is the overflow condition: if the `dividend` is the minimum integer ($-2^{31}$) and the `divisor` is $-1$, the mathematical result is $2^{31}$, which exceeds the maximum representable integer ($2^{31}-1$). This specific scenario must be checked upfront, and the result should be clamped to $2^{31}-1$.

---

### **The Naive Subtraction Approach**

A simple, brute-force approach would be to repeatedly subtract the absolute `divisor` from the absolute `dividend` and count how many times the subtraction occurs before the remaining `dividend` becomes less than the `divisor`. For example, $10 / 3$: $10 - 3 = 7$ (count 1), $7 - 3 = 4$ (count 2), $4 - 3 = 1$ (count 3). The remainder is 1. The result is 3. This approach has a time complexity of $O(Q)$, where $Q$ is the quotient. If the `dividend` is large (e.g., $2^{31}$) and the `divisor` is small (e.g., 1), this approach is extremely slow and will result in a Time Limit Exceeded (TLE) error.

---

### **The Optimal Strategy: Bitwise Shifting and Exponential Search**

To achieve the required logarithmic time complexity, $O(\log Q)$, we must use a technique that mimics the way computers perform division, which is based on **binary subtraction** and **bit manipulation**. This method leverages the fact that any integer can be expressed as a sum of powers of 2.

Instead of subtracting the `divisor` one time, we seek to subtract the **largest possible power of two times the divisor** from the `dividend` in each step. This is essentially an exponential search for the coefficient of the divisor. 

---

### **The Core Algorithm: Repeated Doubling**

1.  We start with a temporary counter, say `temp_divisor`, initialized to the absolute `divisor`, and a magnitude counter, `multiple`, initialized to 1.
2.  We repeatedly double both `temp_divisor` and `multiple` as long as `temp_divisor` is less than or equal to the remaining `dividend`. Doubling is efficiently done using the left bit-shift operator ($\ll 1$).
    * `temp_divisor` doubles: `temp_divisor = temp_divisor << 1` (equivalent to multiplying by 2).
    * `multiple` doubles: `multiple = multiple << 1` (this represents the $2^k$ multiplier).
3.  Once `temp_divisor` exceeds the `dividend`, we know the previous value was the largest power of two times the `divisor` we could subtract.
4.  We add the corresponding `multiple` (the $2^k$) to the total `quotient`.
5.  We subtract the previous `temp_divisor` (the largest power of two times the divisor) from the `dividend`.
6.  We reset `temp_divisor` and `multiple` back to the initial `divisor` and 1, respectively, and repeat the entire process from the reduced `dividend`.

---

### **Implementation Detail: Using Negative Numbers for Range Safety**

A sophisticated implementation often performs all the core calculations using **negative numbers**. The reason is that the range of negative 32-bit signed integers, $[-2^{31}, -1]$, contains one more element than the range of positive integers, $[1, 2^{31}-1]$. If we convert the `dividend` to its positive form, it might overflow if it was originally $-2^{31}$. By working with the negative values (and negating the final result), we prevent this specific internal overflow during the calculation. For instance, to get the absolute value of $X$, we use $-X$ if $X$ is negative, which is safe.

---

### **Complexity Analysis**

The optimal bitwise shifting approach has a logarithmic time complexity. In each iteration of the outer loop, the magnitude of the remaining `dividend` is reduced by at least half, as we subtract the largest possible multiple of the `divisor`. This reduction process is logarithmic with respect to the quotient $Q$. Thus, the time complexity is $O(\log Q)$, or more accurately, $O(\log N)$ where $N$ is the magnitude of the `dividend`. The space complexity is $O(1)$ as only a few variables are used to store the intermediate calculations.